#### Author: Arash Farahani
#### Date: 8/1/2017

### Usage
This program goes over all the Manhattan GBATed DOL-QCEW files and checks the error sheet in Google Maps API to retreive Formatted Addresses and Lat-Lon. The process is repeated twice: first with the original address, and next with the Trade/Legal name added to the address. 

#### The next steps
1. Check the final addresses manually: do the Google Formatted Addresses make sense?
2. Run through GBAT again using a final address.

In [307]:
import logging
logging.basicConfig(filename='Y:/ython Procedures/QCEW_Address_Check.log',level=logging.DEBUG)

import googlemaps
from datetime import datetime
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 60
import re, os
import configparser

config = configparser.RawConfigParser()
config.read('API_Keys.cfg')
google_api_key = config.get('Google', 'QCEW_API_Key')

gmaps = googlemaps.Client(key=google_api_key)

#test
geocode_result=gmaps.geocode("BETH ISRAEL MEDICAL CENTER, 1ST AVE AT 16TH ST, New York, NY 10003")
print(geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'])
#print(geocode_result)


40.7333526 -73.9823138


In [275]:
directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll skip them.
processed_files=['forgbatmn01.xlsx','forgbatmn05.xlsx']

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' not in filename and filename not in processed_files:
        print(os.path.join(directory,filename.encode()))
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(adds.sheet_names)
        df = adds.parse('GBATErr')
        df = df.drop(['BadRecordId','GRC','ReasonCode'], 1)   #drop the GBAT Err Columns
        print(filename, ' was loaded')
        df.head()
        
        
        df.fillna('')
        # trade2 is trade name when available, and legal name when not.
        df['trade2'] = ""
        df.loc[df.trade.fillna('')!= '', 'trade2'] = df.trade 
        df.loc[df.trade.fillna('')== '', 'trade2'] = df.legal
        # trade(or legal) name + Original Address + City, State, Zip
        df['NameAddress']= df.trade2.fillna('')+', '+df.originaladdress.fillna('')+', New York, NY '+df.pzip.apply(str)
        df['Address']= df.originaladdress.fillna('')+', New York, NY '+df.pzip.apply(str)
        df['NameAddress'].head()
        
        #df.reset_index(inplace=True)
        df['Gformatted_address0']= ""
        df['Glat0']= 0
        df['Glon0']= 0
        df['GPartial0']= False
        df['Gtypes0']=""
        df['Gformatted_address1']= ""
        df['Glat1']= 0
        df['Glon1']= 0
        df['GPartial1']= False
        df['Gtypes1']=""
        i=-1
        for var in ['Address','NameAddress']:
            print('Started checking variable ',var)
            i+=1
            for index, row in df.iterrows():
                if index<=len(df.index):
                    geocode_result=gmaps.geocode(row[var])
                    if len(geocode_result)>0:
                        if 'partial_match' in geocode_result:
                            df.loc[df.index == index,['GPartial'+ str(i),'Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                                 geocode_result[0]['partial_match'], geocode_result[0]['formatted_address']
                                                 ,geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'],str(geocode_result[0]['types']))
                        else:
                            df.loc[df.index == index,['Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                geocode_result[0]['formatted_address'],geocode_result[0]['geometry']['location']['lat']
                                ,geocode_result[0]['geometry']['location']['lng'],str(geocode_result[0]['types']))
                    else:
                        df.loc[df.index == index,['Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                                                                                    'Not Found',0,0,str([0]))
            # Prepare for Second Run of GBAT
            df['Gzip'+ str(i)]= 0
            pat2= r".*([0-9]{5}).*"
            repl0 = lambda m: m.group(1)
            df['Gzip1']=df['Gformatted_address'+ str(0)].str.replace(pat, repl0)
            
            pat= r"([0-9\-]+)(.*?)(,.*)"
            repl1 = lambda m: m.group(1)
            repl2 = lambda m: m.group(2)
            df['Gnumber'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl1)
            df['Gstreet'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl2)
            df['boro']=1
            df[['Gnumber'+ str(i),'Gformatted_address'+ str(i)]].head(10)
            
        # Save The Results
        df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
        pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        writer = pd.ExcelWriter(os.path.join(directory,('_GoogOut_'+filename).encode()).decode())
        df.to_excel(writer,'Sheet1')
        print('Processed data and saved: ', '_GoogOut_'+filename )
        #df2.to_excel(writer,'Sheet2')
        writer.save()
    else:
        continue


## SOME OTHER STUFF
# Look up an address with reverse geocoding
#reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit
#now = datetime.now()
#directions_result = gmaps.directions("Sydney Town Hall",
#                                     "Parramatta, NSW",
#                                     mode="transit",
#                                     departure_time=now)

## Some Other Tests
#geocode_result=gmaps.geocode("BETH ISRAEL MEDICAL CENTER, 1ST AVE AT 16TH ST, New York, NY 10003")
#print(geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'])
#print(geocode_result[0]['partial_match'])
#print(geocode_result[0])



b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn07.xlsx'
['forgbatmn07', 'GBATOut', 'GBATErr', 'GBATExt']
forgbatmn07.xlsx  was loaded
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatmn07.xlsx
b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn11.xlsx'
['forgbatmn11', 'GBATOut', 'GBATErr']
forgbatmn11.xlsx  was loaded
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatmn11.xlsx
b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn02.xlsx'
['forgbatmn02', 'GBATOut', 'GBATErr']
forgbatmn02.xlsx  was loaded
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatmn02.xlsx
b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn13.xlsx'
['forgbatmn13', 'GBATOut', 'GBATErr']
forgbatmn13.xlsx  was loaded
Started checking

In [276]:
print(len(df.index))

2610


### Process Results for further cleaning
At the moment (8/2/17) I just check the records that are in my CRP-2005 area zipcodes and have total wages ge 200k.

In [304]:
# Add Google zip-codes to files that were processed previoiusly

directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll edit them.
exclude_files=['_GoogOut_forgbatmn01 - edited.xlsx']

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' in filename and filename not in exclude_files:
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(filename, ' to load')
        print('sheet names: ',adds.sheet_names)        
        df = adds.parse('Sheet1')      
        print(filename, ' was loaded')
        
        for i in [0,1]:
            df['Gzip'+ str(i)]= 0
            pat2= r".*NY ([0-9]{5}).*"
            repl0 = lambda m: m.group(1)
            df['Gzip'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat2, repl0)
        
        
        # Save The Results
        df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
        pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        writer = pd.ExcelWriter(os.path.join(directory,(filename).encode()).decode())
        df.to_excel(writer,'Sheet1')
        print('Processed data and saved: ', filename )
        #df2.to_excel(writer,'Sheet2')
        writer.save()
df.head()


_GoogOut_forgbatmn01.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn01.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn01.xlsx


,pstreet1,streetnumber,streetname,er,run,legal,trade,originaladdress,totwages01a,avgemp01a,...,Glat1,Glon1,GPartial1,Gtypes1,Gnumber0,Gstreet0,Gnumber1,Gstreet1,Gzip0,Gzip1
762,555 W 5TH ST,555,WEST 5 STREET,1851403,0,HEALTHFIRST INC,HEALTHFIRST INC,555 W 5TH ST,33299525,686.000000,...,0.000000,0.000000,False,[0],555,W 5th St,Not Found,Not Found,13126,Not Found
1126,260W 44TH ST,260,44 STREET,3881763,0,MSX INTERNATIONAL ENGINEERING,MSX INTERNATIONAL ENGINEERING,260W 44TH ST,18050605,305.000000,...,0.000000,0.000000,False,[0],260,W 44th St,Not Found,Not Found,10036,Not Found
1294,MORGAN STANLEY VIP,MORGAN,NaN,4507308,6,CORESTAFF ASSOCIATES LLC,CORESTAFF ASSOCIATES LLC,MORGAN STANLEY VIP,12400216,450.666667,...,40.761012,-73.978157,False,"['establishment', 'finance', 'point_of_interest']",31,West 52nd Street,31,West 52nd Street,10019,10019
1904,#5 HANOVER SQUARE 6TH FL,#5,NaN,6370321,9,RELIANCE INSURANCE CO,RELIANCE INSURANCE CO,#5 HANOVER SQUARE 6TH FL,10488659,110.111111,...,0.000000,0.000000,False,[0],5,Hanover Square,Not Found,Not Found,10004,Not Found
1220,DELAWARE INC ETAL,DELAWA,NaN,4292311,0,SANWA FINANCIAL PRODUCTS,SANWA FINANCIAL PRODUCTS,DELAWARE INC ETAL,10301946,16.666667,...,0.000000,0.000000,False,[0],Not Found,Not Found,Not Found,Not Found,Not Found,Not Found


In [305]:
target_zips=[10001,10002,10004,10005,10006,10007,10013,10018,10038,10041,10048,10119,10123]

directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll skip them.
incomplete_files=['_GoogOut_forgbatmn01 - edited.xlsx']

# Clean up the files for processing 

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' in filename and filename not in incomplete_files:
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(filename, ' to load')
        print('sheet names: ',adds.sheet_names)        
        df = adds.parse('Sheet1')
        df['zip_keep'] = (df.Gzip0.isin(target_zips)) | (df.Gzip1.isin(target_zips)) | (df.pzip.isin(target_zips))   #drop the GBAT Err Columns
        print(filename, ' was loaded')
        
        # Save The Results
        df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
        pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        writer = pd.ExcelWriter(os.path.join(directory,(filename).encode()).decode())
        df.to_excel(writer,'Sheet1')
        print('Processed data and saved: ', filename )
        #df2.to_excel(writer,'Sheet2')
        writer.save()


_GoogOut_forgbatmn11.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn11.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn11.xlsx
_GoogOut_forgbatmn06.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn06.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn06.xlsx
_GoogOut_forgbatmn01.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn01.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn01.xlsx
_GoogOut_forgbatmn05.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn05.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn05.xlsx
_GoogOut_forgbatmn13.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn13.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn13.xlsx
_GoogOut_forgbatmn10.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn10.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn10.xlsx
_GoogOut_forgbatmn00.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn00.xlsx  was loa

In [308]:
df.head()

,pstreet1,streetnumber,streetname,er,run,legal,trade,originaladdress,totwages08a,avgemp08a,naics1,naics2,naics3,naics4,pzip,meei,own,boro,recordid,trade2,NameAddress,Address,Gformatted_address0,Glat0,Glon0,GPartial0,Gtypes0,Gformatted_address1,Glat1,Glon1,GPartial1,Gtypes1,Gzip0,Gzip1,Gnumber0,Gstreet0,Gnumber1,Gstreet1,same_GF_Address,zip_keep
0,480 MAPLE STREET,480,MAPLE STREET,450434,109,JBFCS,MAPLE HOUSE,480 MAPLE STREET,481634,14.000000,623220,623220,623220,623220,10018,3,5,1,95,MAPLE HOUSE,"MAPLE HOUSE, 480 MAPLE STREET, New York, NY 10018","480 MAPLE STREET, New York, NY 10018","480 Maple St, Brooklyn, NY 11225, USA",40.661025,-73.944682,False,['premise'],"480 Maple St, Brooklyn, NY 11225, USA",40.661022,-73.944692,False,"['establishment', 'health', 'point_of_interest']",11225,11225,480,Maple St,480,Maple St,True,True
1,PS 154,PS,NaN,451039,4,FOOD BANK FOR NEW YORK CITY,NaN,PS 154,4123,2.000000,0,0,0,624120,10027,3,5,1,280,FOOD BANK FOR NEW YORK CITY,"FOOD BANK FOR NEW YORK CITY, PS 154, New York,...","PS 154, New York, NY 10027","250 W 127th St, New York, NY 10027, USA",40.810470,-73.948712,False,"['establishment', 'point_of_interest', 'school']","250 W 127th St, New York, NY 10027, USA",40.810470,-73.948712,False,"['establishment', 'point_of_interest', 'school']",10027,10027,250,W 127th St,250,W 127th St,True,False
2,PS 134,PS,NaN,451039,5,FOOD BANK FOR NEW YORK CITY,NaN,PS 134,2603,1.000000,0,0,0,624120,10002,3,5,1,281,FOOD BANK FOR NEW YORK CITY,"FOOD BANK FOR NEW YORK CITY, PS 134, New York,...","PS 134, New York, NY 10002","293 E Broadway, New York, NY 10002, USA",40.714301,-73.982966,False,"['establishment', 'point_of_interest', 'school']","39 Broadway, New York, NY 10038, USA",40.706356,-74.013422,False,"['establishment', 'point_of_interest']",10002,10038,293,E Broadway,39,Broadway,False,True
3,522 W 173 ST,522,WEST 173 STREET,452470,9,COMUNILIFE INC,NaN,522 W 173 ST,236506,8.416667,624221,624221,624221,624221,10016,3,5,1,454,COMUNILIFE INC,"COMUNILIFE INC, 522 W 173 ST, New York, NY 10016","522 W 173 ST, New York, NY 10016","522 W 173rd St, New York, NY 10032, USA",40.843289,-73.935492,False,['street_address'],"315 E 148th St, Bronx, NY 10451, USA",40.816325,-73.921244,False,"['establishment', 'health', 'point_of_interest']",10032,10451,522,W 173rd St,315,E 148th St,False,False
4,139 141 AVE D,139,141 AVENUE,454798,12,BRC HUMAN SERVICES CORP,BOWERY RESIDENTS COMMITTEE INC,139-141 AVE D,568347,17.500000,623990,623990,623990,623990,10009,3,5,1,684,BOWERY RESIDENTS COMMITTEE INC,"BOWERY RESIDENTS COMMITTEE INC, 139-141 AVE D,...","139-141 AVE D, New York, NY 10009","141 Avenue D, New York, NY 10009, USA",40.724677,-73.975626,False,['street_address'],"139 Avenue D, New York, NY 10009, USA",40.724684,-73.975540,False,"['establishment', 'health', 'point_of_interest']",10009,10009,141,Avenue D,139,Avenue D,False,False
